In [ ]:
!pip install requests
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import re
from datetime import datetime, date
from pprint import pprint
from urllib.parse import urljoin, urlparse, urlunparse

import requests
from bs4 import BeautifulSoup

In [ ]:
filmarks_url = urlunparse(("https", "filmarks.com", "", "", "", ""))
filmarks_url

'https://filmarks.com'

In [ ]:
filmarks_id = "fairy26"

In [ ]:
urljoin(filmarks_url, f"/users/{filmarks_id}")

'https://filmarks.com/users/fairy26'

In [ ]:
# 色々試す

urlparse("https://filmarks.com/users/fairy26")
# ParseResult(scheme='https', netloc='filmarks.com', path='/users/fairy26', params='', query='', fragment='')

urlunparse(("https", "filmarks.com", "/users/fairy26", "", "", ""))
# https://filmarks.com/users/fairy26

urlparse("/movies/33172?mark_id=147182566").path
# /movies/33172

'/movies/33172'

In [ ]:
res = requests.get(urljoin(filmarks_url, f"/users/{filmarks_id}"))
soup = BeautifulSoup(res.text, "html.parser")

In [ ]:
# ページ数
num_pages = 1
regex = re.compile(f"/users/{filmarks_id}\?page=(\d+)")
pagination_last = soup.find("a", class_="c-pagination__last")["href"]
result = regex.match(pagination_last)
if result:
  num_pages = int(result.group(1))
print(num_pages)

7


In [ ]:
parser = "html.parser"
for num in range(1, num_pages+1):
  url = urljoin(filmarks_url, f"/users/{filmarks_id}?page={num}")
  
  # ユーザーページnum番目
  res = requests.get(url)
  if res.status_code != 200:
    continue
  soup = BeautifulSoup(res.text, parser)

  for card_title in soup.find_all("h3", class_="c-content-card__title"):
    movie_path_query = card_title.find("a")["href"]
    movie_url = urljoin(filmarks_url, movie_path_query)
    print(f"{movie_url = }")

    # 映画ページ
    res = requests.get(movie_url)
    if res.status_code != 200:
      continue
    soup_movie = BeautifulSoup(res.text, parser)

    ## 映画のデータ
    detail = soup_movie.find("div", class_="p-content-detail__body")
    other_info = list(detail.find("div", class_="p-content-detail__other-info").children)
    people_list_others = list(detail.find("div", class_="p-content-detail__people-list-others__wrapper").children)

    # タイトル
    title: str = detail.find("h2", class_="p-content-detail__title").find("span").text
    # 画像url
    img_url: str = detail.find("img")["src"]
    # 上映日
    release_date: date = datetime.strptime(other_info[0].text, "上映日：%Y年%m月%d日").date()
    # 制作国
    countries: list = [country.text for country in other_info[2].find_all("a")]
    # ジャンル
    genres: list = [genre.text for genre in detail.find("div", class_="p-content-detail__genre").find_all("a")]
    # 監督
    directors: list = [person.text for person in people_list_others[0].find_all("a")]
    # 脚本
    writers: list = [person.text for person in people_list_others[1].find_all("a")]
    # 出演者
    casts: list = [cast.text for cast in detail.find("div", id="js-content-detail-people-cast").find_all("a")]

    ## レビューのデータ
    card_review = soup_movie.find("div", class_="p-mark")
    
    # 鑑賞日
    watch_date: date = datetime.strptime(card_review.find("time")["datetime"], "%Y-%m-%d %H:%M").date()
    # スコア
    score = float(card_review.find("div", class_="c-rating__score").text)
    # 感想
    review = card_review.find("div", class_="p-mark__review").text


    color = "gray"
    if score <= 1.0:
      color = "lightgray"
    elif 1.0 < score <= 2.0:
      color = "brown"
    elif 2.0 < score <= 3.0:
      color = "yellow"
    elif 3.0 < score <= 4.0:
      color = "orange"
    elif 4.0 < score <= 5.0:
      color = "red"

    print(f"{title = }")
    print(f"{img_url = }")
    print(f"{release_date = }")
    print(f"{countries = }")
    print(f"{genres = }")
    print(f"{directors = }")
    print(f"{writers = }")
    print(f"{casts = }")
    print(f"{watch_date = }")
    print(f"{score = }")
    print(f"{review = }")
    print(f"{color = }")
    break
  break

movie_url = 'https://filmarks.com/movies/33172?mark_id=147182566'
title = 'タクシードライバー'
img_url = 'https://d2ueuvlup6lbue.cloudfront.net/attachments/8faceb5f89e1587fae260af6286cc7ff2fab793f/store/fitpad/260/364/088162cd4d520a301a9a4b14b65073c64b4600a325c90afaee131dcb416a/_.jpg'
release_date = datetime.date(1976, 9, 18)
countries = ['アメリカ']
genres = ['サスペンス', 'クライム', 'スリラー']
directors = ['マーティン・スコセッシ']
writers = ['ポール・シュレイダー']
casts = ['ロバート・デ・ニーロ', 'シビル・シェパード', 'ピーター・ボイル', 'ジョディ・フォスター', 'アルバート・ブルックス', 'ハーヴェイ・カイテル', 'ジョー・スピネル', 'マーティン・スコセッシ', 'ダイアン・アボット', 'ヴィクター・アルゴ', 'レオナルド・ハリス']
watch_date = datetime.date(2023, 1, 13)
score = 3.7
review = 'バックボーン違いすぎてアメリカンニューシネマは理解共感しづらい'
color = 'orange'


# Notion

In [ ]:
db_id = "75fdbc514ccb4805bf3776a7f7a1b03b"
notion_integration_secret_token = 'secret_Wjz2Po6ZbgpOIIeMJYjLVvtCG0O6iBjDLRldd9z3n9W'
api_url = "https://api.notion.com/v1/"

## DB情報のGET

In [ ]:
headers = {
    "Authorization": f"Bearer {notion_integration_secret_token}",
    "Notion-Version": "2021-08-16",
}

r = requests.get(urljoin(api_url, f"databases/{db_id}"), headers=headers)

In [ ]:
pprint(r.json())

{'archived': False,
 'cover': None,
 'created_by': {'id': '2fb3fd47-a66e-4de0-8336-e88584cc51b8', 'object': 'user'},
 'created_time': '2023-01-14T11:58:00.000Z',
 'description': [],
 'icon': None,
 'id': '75fdbc51-4ccb-4805-bf37-76a7f7a1b03b',
 'is_inline': True,
 'last_edited_by': {'id': '2fb3fd47-a66e-4de0-8336-e88584cc51b8',
                    'object': 'user'},
 'last_edited_time': '2023-01-14T13:44:00.000Z',
 'object': 'database',
 'parent': {'page_id': '19c8b3b0-0d39-48a1-80a1-117f49cac18d',
            'type': 'page_id'},
 'properties': {'filmarks': {'id': 'J%3FGu',
                             'name': 'filmarks',
                             'type': 'url',
                             'url': {}},
                'ジャンル': {'id': 'u%3DWK',
                         'multi_select': {'options': [{'color': 'gray',
                                                       'id': '04af1892-f59f-4753-8f3a-ba1db250fd8a',
                                                       'name': 'サスペンス'}

## DBの全ページを取得(POST)

In [ ]:
headers = {
    "Authorization": f"Bearer {notion_integration_secret_token}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

r = requests.post(urljoin(api_url, f"databases/{db_id}/query"), headers=headers)

In [ ]:
pprint(r.json()["results"][1])

{'archived': False,
 'cover': {'external': {'url': 'https://d2ueuvlup6lbue.cloudfront.net/attachments/6c63b649e8a5217ff171a3f8902c31a7802a7f5c/store/fitpad/260/364/59bc12e9f969df0a664a685f90793efe7b3d2dc88e6793fde61aaf71feee/_.jpg'},
           'type': 'external'},
 'created_by': {'id': '2fb3fd47-a66e-4de0-8336-e88584cc51b8', 'object': 'user'},
 'created_time': '2023-01-14T11:58:00.000Z',
 'icon': {'external': {'url': 'https://www.notion.so/icons/movie_orange.svg'},
          'type': 'external'},
 'id': '342bc15f-0170-402a-8c2f-73f09e012ef2',
 'last_edited_by': {'id': '2fb3fd47-a66e-4de0-8336-e88584cc51b8',
                    'object': 'user'},
 'last_edited_time': '2023-01-14T14:28:00.000Z',
 'object': 'page',
 'parent': {'database_id': '75fdbc51-4ccb-4805-bf37-76a7f7a1b03b',
            'type': 'database_id'},
 'properties': {'filmarks': {'id': 'J%3FGu',
                             'type': 'url',
                             'url': 'https://filmarks.com/movies/61639?mark_id=1470197

In [ ]:
pbar_page_cache = {}

db_pbar_id = "01328a1886924dd9b02ef3c41c37426e"
headers = {
    "Authorization": f"Bearer {notion_integration_secret_token}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

def get_pbar_page_id(year: int):
  if year in pbar_page_cache:
    return pbar_page_cache[year]

  data = {
    "filter": {
        "property": "年初",
        "date": {"equals": date(year, 1, 1).isoformat()}
    }
  }

  r = requests.post(urljoin(api_url, f"databases/{db_pbar_id}/query"), headers=headers, data=json.dumps(data))

  if r.status_code == 200 and r.json()["results"] != []:
    id = r.json()["results"][0]["id"]
    pbar_page_cache[year] = id

    return id

  data = {
      'parent': {'database_id': db_pbar_id},
      'properties': {
          '年': {'title': [{'text': {'content': str(year)}}]},
          '年初': {'date': {'start': date(year, 1, 1).isoformat()}}
          },
  }

  r = requests.post(urljoin(api_url, "pages"), headers=headers, data=json.dumps(data))
  if r.status_code == 200:
    id = r.json()["id"]
    pbar_page_cache[year] = id

    return id

  return ""

In [ ]:
pprint(r.json()["results"][0])

{'archived': False,
 'cover': None,
 'created_by': {'id': '2fb3fd47-a66e-4de0-8336-e88584cc51b8', 'object': 'user'},
 'created_time': '2023-01-14T09:06:00.000Z',
 'icon': None,
 'id': '8b3e3291-90af-4e52-80a4-1b75fb0e83b0',
 'last_edited_by': {'id': '2fb3fd47-a66e-4de0-8336-e88584cc51b8',
                    'object': 'user'},
 'last_edited_time': '2023-01-14T12:44:00.000Z',
 'object': 'page',
 'parent': {'database_id': '01328a18-8692-4dd9-b02e-f3c41c37426e',
            'type': 'database_id'},
 'properties': {'年': {'id': 'title',
                      'title': [{'annotations': {'bold': False,
                                                 'code': False,
                                                 'color': 'default',
                                                 'italic': False,
                                                 'strikethrough': False,
                                                 'underline': False},
                                 'href': None,
          

In [ ]:
year = 2022
data = {
    'parent': {
        'database_id': '01328a18-8692-4dd9-b02e-f3c41c37426e'
        },
    'properties': {
        '年': {'title': [{'text': {'content': str(year)}}]},
        '年初': {'date': {'start': f"{year}-01-01"}}
        },
}

## ページのCREATE

```bash
curl 'https://api.notion.com/v1/pages' \
  -H 'Authorization: Bearer '"$NOTION_API_KEY"'' \
  -H "Content-Type: application/json" \
  -H "Notion-Version: 2022-06-28" \
  --data '{
	"parent": { "database_id": "d9824bdc84454327be8b5b47500af6ce" },
  "icon": {
  	"emoji": "🥬"
  },
	"cover": {
		"external": {
			"url": "https://upload.wikimedia.org/wikipedia/commons/6/62/Tuscankale.jpg"
		}
	},
	"properties": {
		"Name": {
			"title": [
				{
					"text": {
						"content": "Tuscan Kale"
					}
				}
			]
		},
		"Description": {
			"rich_text": [
				{
					"text": {
						"content": "A dark green leafy vegetable"
					}
				}
			]
		},
		"Food group": {
			"select": {
				"name": "Vegetable"
			}
		},
		"Price": { "number": 2.5 }
	},
	"children": [
		{
			"object": "block",
			"type": "heading_2",
			"heading_2": {
				"rich_text": [{ "type": "text", "text": { "content": "Lacinato kale" } }]
			}
		},
		{
			"object": "block",
			"type": "paragraph",
			"paragraph": {
				"rich_text": [
					{
						"type": "text",
						"text": {
							"content": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
							"link": { "url": "https://en.wikipedia.org/wiki/Lacinato_kale" }
						}
					}
				]
			}
		}
	]
}'
```

In [ ]:
headers = {
    "Authorization": f"Bearer {notion_integration_secret_token}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

data = {
    "parent": {
        "database_id": db_id,
    },
    "icon": {
        'external': {'url': f"https://www.notion.so/icons/movie_{color}.svg"},
    },
    "cover": {
        'external': {'url': img_url},
    },
    "properties": {
        'タイトル': {
            'title': [
                {
                    'text': {
                        'content': title
                    }
                }
            ],
        },
        'スコア': {
            'number': score
        },
        '感想': {
            'rich_text': [
                {
                    "type": "text",
                    'text': {
                        'content': review
                    },
                 }
            ],
        },
        '鑑賞日': {
            'date': {
                "start": watch_date.isoformat()
            }
        },
        '上映日': {
            'date': {
                'start': release_date.isoformat()
            }
        },
        '制作国': {
            'multi_select': [{'name': country} for country in countries],
        },
        'ジャンル': {
            'multi_select': [{'name': genre} for genre in genres],
        },
        '監督': {
            'multi_select': [{'name': person} for person in directors],
        },
        '脚本': {
            'multi_select': [{'name': person} for person in writers],
        },
        '出演者': {
            'multi_select': [{'name': person} for person in casts],
        },
        'filmarks': {'url': movie_url},
        '画像': {'url': img_url}
    }
}

relation_id = get_pbar_page(watch_date.year)
if relation_id:
  data["properties"]['集計']= {
      'relation': [{"id": relation_id}],
      'has_more': False,
      }

r = requests.post(urljoin(api_url, "pages"), headers=headers, data=json.dumps(data))
print(r.status_code)

200


In [ ]:
pprint(r.json())

{'archived': False,
 'cover': {'external': {'url': 'https://d2ueuvlup6lbue.cloudfront.net/attachments/8faceb5f89e1587fae260af6286cc7ff2fab793f/store/fitpad/260/364/088162cd4d520a301a9a4b14b65073c64b4600a325c90afaee131dcb416a/_.jpg'},
           'type': 'external'},
 'created_by': {'id': 'fc670814-e99b-4092-99f3-715cbc098986', 'object': 'user'},
 'created_time': '2023-01-14T16:03:00.000Z',
 'icon': {'external': {'url': 'https://www.notion.so/icons/movie_orange.svg'},
          'type': 'external'},
 'id': '7726cce9-212a-4634-a6b3-d78f1a222bd8',
 'last_edited_by': {'id': 'fc670814-e99b-4092-99f3-715cbc098986',
                    'object': 'user'},
 'last_edited_time': '2023-01-14T16:03:00.000Z',
 'object': 'page',
 'parent': {'database_id': '75fdbc51-4ccb-4805-bf37-76a7f7a1b03b',
            'type': 'database_id'},
 'properties': {'Created by': {'created_by': {'avatar_url': None,
                                              'bot': {},
                                              'id': 

# 全部まとめる

In [44]:
import json
import re
from datetime import datetime, date
from pprint import pprint
from urllib.parse import urljoin, urlparse, urlunparse

import requests
from bs4 import BeautifulSoup

filmarks_url = "https://filmarks.com"
filmarks_id = "fairy26"

NOTION_TOKEN = 'secret_Wjz2Po6ZbgpOIIeMJYjLVvtCG0O6iBjDLRldd9z3n9W'
DB_MOVIE_ID = "75fdbc514ccb4805bf3776a7f7a1b03b"
DB_PBAR_ID = "01328a1886924dd9b02ef3c41c37426e"

API_URL = "https://api.notion.com/v1/"
HEADERS = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

pbar_page_cache = {}


def get_pbar_page_id(year: int):
  if year in pbar_page_cache:
    return pbar_page_cache[year]

  data = {
    "filter": {
        "property": "年初",
        "date": {"equals": date(year, 1, 1).isoformat()}
    }
  }

  r = requests.post(urljoin(API_URL, f"databases/{DB_PBAR_ID}/query"), headers=HEADERS, data=json.dumps(data))

  if r.status_code == 200 and r.json()["results"] != []:
    id = r.json()["results"][0]["id"]
    pbar_page_cache[year] = id

    return id

  data = {
      'parent': {'database_id': DB_PBAR_ID},
      'properties': {
          '年': {'title': [{'text': {'content': str(year)}}]},
          '年初': {'date': {'start': date(year, 1, 1).isoformat()}}
          },
  }

  r = requests.post(urljoin(API_URL, "pages"), headers=HEADERS, data=json.dumps(data))
  if r.status_code == 200:
    id = r.json()["id"]
    pbar_page_cache[year] = id

    return id

  return ""


class Properties:
  def __init__(self, db_id):
    self.db_id = db_id

    self.title = ""
    self.score = 0.0
    self.review = ""
    
    self.movie_url = ""
    self.img_url = ""
    self.watch_date: date = None
    self.release_year: int = None
    self.countries = []
    self.genres = []
    self.directors = []
    self.writers = []
    self.casts = []
  

  def export(self) -> dict:
    def _multi_select_items(name_llist):
      return [{'name': name} for name in name_llist]
    
    def _color_of(score):
      if not isinstance(score, float):
        return "gray"
      
      star = round(score)
      if star <= 1: # 0.0 <= score < 1.5
        return "lightgray"
      elif star == 2: # 1.5 <= score < 2.5
        return "brown"
      elif star == 3: # 2.5 <= score < 3.5
        return "yellow"
      elif star == 4: # 3.5 <= score < 4.5
        return "orange"
      elif star == 5: # 4.5 <= score < 5.0
        return "red"
      
      return "gray"

    return {
        "parent": {"database_id": self.db_id},
        "icon": {'external': {'url': f"https://www.notion.so/icons/movie_{_color_of(self.score)}.svg"}},
        "cover": {'external': {'url': self.img_url}},
        "properties": {
          'タイトル': {'title': [{'text': {'content': self.title}}]},
          'スコア': {'number': self.score},
          '感想': {'rich_text': [{'text': {'content': self.review}}]},
          '鑑賞日': {'date': {"start": self.watch_date.isoformat()}},
          '上映年': {'number': self.release_year},
          '制作国': {'multi_select': _multi_select_items(self.countries)},
          'ジャンル': {'multi_select': _multi_select_items(self.genres)},
          '監督': {'multi_select': _multi_select_items(self.directors)},
          '脚本': {'multi_select': _multi_select_items(self.writers)},
          '出演者': {'multi_select': _multi_select_items(self.casts)},
          'filmarks': {'url': self.movie_url},
          '画像': {'url': self.img_url},
          '集計': {
            'relation': [{"id": get_pbar_page_id(self.watch_date.year)}],
            'has_more': False,
            }
          }
        }
    

  def parse(self, movie_url: str) -> None:
    self.movie_url = movie_url
    res = requests.get(movie_url)
    if res.status_code != 200:
      return
    
    soup = BeautifulSoup(res.text, parser)

    # 映画のデータ
    detail = soup.find("div", class_="p-content-detail__body")
    other_info = list(detail.find("div", class_="p-content-detail__other-info").children)
    people_list_others = list(detail.find("div", class_="p-content-detail__people-list-others__wrapper").children)

    self.title = detail.find("h2", class_="p-content-detail__title").find("span").text
    self.img_url = detail.find("img")["src"]
    try:
      self.release_year = datetime.strptime(other_info[0].text, "上映日：%Y年%m月%d日").year
      self.countries = [country.text for country in other_info[2].find_all("a")]
    except ValueError as e:
      # https://filmarks.com/movies/55537 のように上映日情報がない映画もある
      self.release_year = int(detail.find("h2", class_="p-content-detail__title").find("a").text[:-1])
      self.countries = [country.text for country in other_info[1].find_all("a")]
    self.genres = [genre.text for genre in detail.find("div", class_="p-content-detail__genre").find_all("a")]
    self.directors = [person.text for person in people_list_others[0].find_all("a")]
    try:
      self.writers = [person.text for person in people_list_others[1].find_all("a")]
    except IndexError as e:
      # https://filmarks.com/movies/80435 のように脚本情報がない映画もある
      pass
    try:
      self.casts = [cast.text for cast in detail.find("div", id="js-content-detail-people-cast").find_all("a")]
    except AttributeError as e:
      # https://filmarks.com/movies/86613 のように出演者情報がない映画もある
      pass

    # レビューのデータ
    card_review = soup.find("div", class_="p-mark")
    
    ## 鑑賞日
    self.watch_date = datetime.strptime(card_review.find("time")["datetime"], "%Y-%m-%d %H:%M").date()
    ## スコア
    self.score = float(card_review.find("div", class_="c-rating__score").text)
    ## 感想
    self.review = card_review.find("div", class_="p-mark__review").text

In [46]:
def _color_of(score):
  if not isinstance(score, float):
    return "gray"
  
  star = round(score)
  if star <= 1: # 0.0 <= score < 1.5
    return "lightgray"
  if star == 2: # 1.5 <= score < 2.5
    return "brown"
  if star == 3: # 2.5 <= score < 3.5
    return "yellow"
  if star == 4: # 3.5 <= score < 4.5
    return "orange"
  if star == 5: # 4.5 <= score < 5.0
    return "red"

In [36]:
from IPython.core import payload
# ------------------------------------------------------------------------------------------------------------
# すでに記録してあるページをキャッシュする
# ------------------------------------------------------------------------------------------------------------
existing_page = {} # dict[str, date] 
# 存在確認用
# 同じタイトルもあると思われるため、タイトルと鑑賞日で判定する

pages = []
payload = {"page_size": 100} # Max: 100
while True:
  r = requests.post(urljoin(API_URL, f"databases/{DB_MOVIE_ID}/query"), headers=HEADERS, data=json.dumps(payload))
  pages += r.json()['results']

  if r.status_code != 200:
    break

  if r.json()['has_more']:
    payload['start_cursor'] = r.json()['next_cursor']
    continue

  break

for page in pages:
  title = page['properties']['タイトル']['title'][0]['plain_text']
  watch_date = page['properties']['鑑賞日']['date']
  if watch_date:
    watch_date = datetime.fromisoformat(watch_date['start']).date()
  existing_page[title] = watch_date


def exists(title, watch_date) -> bool:
  return title in existing_page and watch_date == existing_page[title]

In [6]:
# ------------------------------------------------------------------------------------------------------------
# ページ数
# ------------------------------------------------------------------------------------------------------------
num_pages = 1
res = requests.get(urljoin(filmarks_url, f"/users/{filmarks_id}"))
soup = BeautifulSoup(res.text, "html.parser")
pagination_last = soup.find("a", class_="c-pagination__last")["href"]
result = re.match(f"/users/{filmarks_id}\?page=(\d+)", pagination_last)
if result:
  num_pages = int(result.group(1))

In [52]:

# ------------------------------------------------------------------------------------------------------------
# 全レビューを読み取る
# ------------------------------------------------------------------------------------------------------------
parser = "html.parser"
reviews = []
for num in range(1, num_pages+1):
  # ユーザーページnum番目
  res = requests.get(urljoin(filmarks_url, f"/users/{filmarks_id}?page={num}"))
  if res.status_code != 200:
    continue
  
  soup = BeautifulSoup(res.text, parser)

  for card_title in soup.find_all("h3", class_="c-content-card__title"):
    movie_path_query = card_title.find("a")["href"]
    movie_url = urljoin(filmarks_url, movie_path_query)

    print(f"■ {movie_url}") # debug

    prop = Properties(DB_MOVIE_ID)
    prop.parse(movie_url)
    
    print(prop.title) # debug
    if exists(prop.title, prop.watch_date):
      print('登録されています')
      break
    
    reviews.append(prop.export())
    print()
  else:
    continue
  break

■ https://filmarks.com/movies/100914?mark_id=147373499
フォトコピー

■ https://filmarks.com/movies/20037?mark_id=147350626
ラスベガスをぶっつぶせ

■ https://filmarks.com/movies/69390?mark_id=147348059
新感染 ファイナル・エクスプレス

■ https://filmarks.com/movies/24065?mark_id=147330793
ステイ・フレンズ

■ https://filmarks.com/movies/33172?mark_id=147182566
タクシードライバー
登録されています


In [53]:
# ------------------------------------------------------------------------------------------------------------
# Notionのページ作成
# ------------------------------------------------------------------------------------------------------------
ng_data = []
for pyload in reviews:
  title = pyload["properties"]['タイトル']['title'][0]['text']['content']
  watch_date = pyload['properties']['鑑賞日']['date']['start']

  if exists(title, watch_date):
    # TODO: 存在すれば更新処理に変更
    print(f"[WARN] 「{title}」のページは存在します")
    break # 以降は記録済みなのでここで終了

  r = requests.post(urljoin(API_URL, "pages"), headers=HEADERS, data=json.dumps(pyload))

  if r.status_code != 200:
    ng_data.append(data)
    print(f"[ERROR] 「{title}」のレビューを作成できませんでした")
    continue
  
  print(f"[INFO] 「{title}」のページを作成しました (id: {r.json()['id']})")

[INFO] 「フォトコピー」のページを作成しました (id: dc37ddab-2acc-4b75-842b-7869924f89bc)
[INFO] 「ラスベガスをぶっつぶせ」のページを作成しました (id: 452041ef-3f99-4a0d-a16c-14d47d5068e2)
[INFO] 「新感染 ファイナル・エクスプレス」のページを作成しました (id: 63945d8b-d32b-400e-800e-55abdd9ccc9d)
[INFO] 「ステイ・フレンズ」のページを作成しました (id: 60b346e0-c96d-464d-86eb-aa58d7702ee1)
